In [1]:
import numpy as np

In [2]:
# load data

# LF input and output
x_l1 = np.loadtxt('./narrow/matter_power_564_Box1000_Part750_15_Box1000_Part3000_z0/train_input_fidelity_0.txt')
y_l1 = np.loadtxt('./narrow/matter_power_564_Box1000_Part750_15_Box1000_Part3000_z0/train_output_fidelity_0.txt')
k_l1 = np.loadtxt('./narrow/matter_power_564_Box1000_Part750_15_Box1000_Part3000_z0/kf.txt')

x_l2 = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/train_input_fidelity_0.txt')
y_l2 = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/train_output_fidelity_0.txt')
k_l2 = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/kf.txt')

# HF input and output
x_h = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/train_input_fidelity_1.txt')
y_h = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/train_output_fidelity_1.txt')
k_h = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/kf.txt')

save_dir = 'N_LHr_stitch_z0'

In [3]:
# loop over the L1 input and find the ones (indices) that are in the HF input
indices = []
for i in range(len(x_l1)):
    if x_l1[i] in x_h:
        indices.append(i)


In [4]:
indices

[144, 145, 146, 168, 169, 170, 195, 196, 197, 204, 205, 206, 336, 337, 338]

In [5]:
k_h

array([-1.59976085, -1.56002463, -1.5202884 , -1.48055217, -1.44081595,
       -1.40107972, -1.36134349, -1.32160727, -1.28187104, -1.24213481,
       -1.20239859, -1.16266236, -1.12292613, -1.08318991, -1.04345368,
       -1.00371745, -0.96398123, -0.924245  , -0.88450877, -0.84477255,
       -0.80503632, -0.76530009, -0.72556387, -0.68582764, -0.64609141,
       -0.60635519, -0.56661896, -0.52688273, -0.48714651, -0.44741028,
       -0.40767406, -0.36793783, -0.3282016 , -0.28846538, -0.24872915,
       -0.20899292, -0.1692567 , -0.12952047, -0.08978424, -0.05004802,
       -0.01031179,  0.02942444,  0.06916066,  0.10889689,  0.14863312,
        0.18836934,  0.22810557,  0.2678418 ,  0.30757802,  0.34731425,
        0.38705048,  0.4267867 ,  0.46652293,  0.50625916,  0.54599538,
        0.58573161,  0.62546784,  0.66520406,  0.70494029,  0.74467652,
        0.78441274,  0.82414897,  0.8638852 ,  0.90362142])

In [6]:
def cut_and_stitch(k_l1, y_l1, k_l2, y_l2, k_h): # y_l1 and y_l2 are 2D arrays
    # index of middle k value
    middle = len(k_h) // 2
    # interpolate the L1 output to the first half HF k values
    y_l1_interp = np.array([np.interp(k_h[:middle], k_l1, y_l1[i,:]) for i in range(len(y_l1))])
    # interpolate the L1 output to the second half HF k values
    y_l2_interp = np.array([np.interp(k_h[middle:], k_l2, y_l2[i,:]) for i in range(len(y_l2))])
    # stitch the two interpolated outputs together
    y_interp = np.concatenate((y_l1_interp, y_l2_interp), axis=1)
    return y_interp

In [7]:
# cut y_l1 and y_l2 and stitch them together
y_lf = cut_and_stitch(k_l1, y_l1[indices], k_l2, y_l2[indices], k_h)

In [8]:
# create dataset for LF-HF correlation
# input: x_h, y_l1[indices], y_l2[indices] (concatenate)
x_xylf = np.concatenate((x_h, y_lf), axis=1)

# output: y_h 
y_h = y_h

In [9]:
# ratio
rat = y_h / y_lf

In [10]:
# save the dataset
# mkdir if not exist
import os
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

np.savetxt(save_dir + '/train_input.txt', x_h)
np.savetxt(save_dir + '/train_output.txt', rat)

# save k_h
np.savetxt(save_dir + '/kf.txt', k_h)